# Testing

MIT License

Copyright (c) 2023 - J.R.Verbiest

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [ ]:
import os

import ei_helper
import helper

import numpy as np
import pandas as pd
import tensorflow as tf

import pingouin as pg

%load_ext autoreload
%autoreload 2

---

## Testing - int8 model

In [ ]:
path_testing = '<home/user/data/mlate2dge/processed/testing/>' # should be moved to a env file
path_model   = '<home/user/data/mlate2dge/artifacts/model/tensorflow_lite_int8_model>' # should be moved to a env file

config_dataset_signal_length = 648 # should be moved to a config file

print("path files: %s", path_testing)
print("path model: %s", path_model)

In [ ]:
files = []

for file in os.listdir(path_testing):
    if file.endswith(".json"):
        files.append(file)

# load tflite model
interpreter = tf.lite.Interpreter(path_model)

# get input and output tensors
input_details  = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# allocate tensors
interpreter.allocate_tensors()

input_type = input_details[0]['dtype']
input_scale, input_zero_point = input_details[0]['quantization']

output_type = output_details[0]['dtype']
output_scale, output_zero_point = output_details[0]['quantization']

print("input details: %s", input_type)
print("input scale: %s", input_scale)
print("input zero_point: %s", input_zero_point)

print("output details: %s", output_type)
print("output scale: %s", output_scale)
print("output zero_point: %s", output_zero_point)

results = []
for file in files:
    X_testing, Y_testing = helper.json_file_to_npy_array(os.path.join(path_testing,file), config_dataset_signal_length)
    if input_type == np.int8:
        # https://www.tensorflow.org/lite/performance/post_training_quantization, equation 
        X_testing = np.around((X_testing/input_scale)+input_zero_point)

    # convert to type given by input_details[0]['dtype']
    X_testing = np.array([X_testing], dtype=input_type)
    
    # create input tensor
    interpreter.set_tensor(input_details[0]['index'], X_testing)
    
    # run inference
    interpreter.invoke()
        
    Y = interpreter.get_tensor(output_details[0]['index'])
    if output_type == np.int8:
        Y = output_scale * (Y.astype(np.float32) - output_zero_point)
    
    results.append([Y[0][0], float(Y_testing)])
        
df = pd.DataFrame(results, columns = ['result', 'expected result'])
df.head()

In [ ]:
gold_standard = 'OptoGait stride length [cm]'
new_method    = 'Model (int8) stride length [cm]'

df.rename(columns = {'expected result':gold_standard, 'result':new_method}, inplace = True)
helper.outcome_metrics(df=df, gold_standard = gold_standard, new_method = new_method, model= 'ei_int8')
df.head()

---

## Testing - float32 (From Edge Impulse Studio)

In [ ]:
ei_api_key    = "<your_ei_api_key>" # should be moved to a env file
ei_project_id = "<your_project_id>" # should be moved to a env file

df_testing_results = ei_helper.model_testing_result_tbl(ei_api_key, ei_project_id)
df = df_testing_results.copy()

gold_standard = 'OptoGait stride length [cm]'
new_method    = 'ML-model (float32) stride length [cm]'

df.rename(columns = {'expected result':gold_standard, 'result':new_method}, inplace = True)
helper.outcome_metrics(df=df, gold_standard = gold_standard, new_method = new_method, model='ei_float32')
df[[gold_standard, new_method]].corr(method='pearson')

df_testing_results.head()